# IBM Applied Data Science Capstone Course by Coursera
## Week 5 Final Report
## Opening a New Restaurant in Lagos, Nigeria.
In this notebook I will determine the best neigborhood or cluster of neiborhoods to open up a restaurant in Lagos state with minimum or no competion from other Restaurants.

First I Build a dataframe of neighborhoods in Lagos,
Get the geographical coordinates of the neighborhoods
Obtain the venue data for the neighborhoods from Foursquare API
Explore and cluster the neighborhoods into regions saturated,moderately saturated and not with restaurants 
Select the best neigborhoods to open a new restaurant in.

### 1. Import Required libraries

In [14]:
import pandas as pd
import geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium


### 2. Copy Neigborhood data from Excel to pandas dataframe

In [15]:
# create a pandas dataframe from the excel
df = pd.read_excel(r'C:\Users\Qudus\Downloads\neigborhoods in lagos.xlsx')
df.head()

,Lagos state lga
0,Eti Osa
1,Kosofe
2,Shibiri Ekune
3,Olojo
4,Okokomaiko


In [16]:
# print the number of rows of the dataframe
df.shape

(115, 1)

### 3. Get the geographical coordinates

In [17]:
# define a function to get coordinates
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Lagos, Nigeria'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [19]:
# use the function to get the coordinates
coords = [ get_latlng(neighborhood) for neighborhood in df["Lagos state lga"].tolist() ]

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


In [21]:
coords
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the two dataframe into one
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(df.shape)
df

(115, 3)


,Lagos state lga,Latitude,Longitude
0,Eti Osa,6.466680,3.583260
1,Kosofe,6.599990,3.415090
2,Shibiri Ekune,6.454710,3.388760
3,Olojo,6.458053,3.203222
4,Okokomaiko,6.464210,3.182200
5,Ojo,6.462620,3.166960
6,Ira,6.650520,3.640000
7,Alaba,6.530317,3.346667
8,Ajangbadi ikemba,6.454710,3.388760
9,Owutu,6.454710,3.388760


In [22]:
# save the DataFrame as CSV file
df.to_csv("df.csv", index=False)

### 5. Use the Foursquare API to explore the neighborhoods and venues in the neigborhoods

In [79]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20191003'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [24]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Lagos state lga']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [26]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df

(1457, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Eti Osa,6.466680,3.583260,kojima kyoto,6.472799,3.568132,Sushi Restaurant
1,Eti Osa,6.466680,3.583260,Domino's pizza Ajah,6.469620,3.586121,Fast Food Restaurant
2,Eti Osa,6.466680,3.583260,Tripps,6.466426,3.588519,Fast Food Restaurant
3,Eti Osa,6.466680,3.583260,Ajah New Garages,6.469428,3.578555,Bus Station
4,Eti Osa,6.466680,3.583260,God is Good Motors,6.471043,3.579403,Bus Station
5,Eti Osa,6.466680,3.583260,Legend bar & saloon,6.473481,3.574591,Salon / Barbershop
6,Eti Osa,6.466680,3.583260,Ajah Market,6.468307,3.565298,Convenience Store
7,Eti Osa,6.466680,3.583260,Ajah,6.469079,3.568777,Bus Station
8,Kosofe,6.599990,3.415090,Mile 12,6.594098,3.405009,Metro Station
9,Kosofe,6.599990,3.415090,Novel bus stop,6.608888,3.426759,Bus Station


In [27]:
# save the DataFrame as CSV file
venues_df.to_csv("venues.csv", index=False)

In [28]:
#to check how many venues per neighborhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Abarenji Street,23,23,23,23,23,23
Abuke Ijeha,12,12,12,12,12,12
Abule Egba,5,5,5,5,5,5
Abule Okuta,8,8,8,8,8,8
Abule abon,7,7,7,7,7,7
Adeniran Ogunsanya,25,25,25,25,25,25
Agbelekale,3,3,3,3,3,3
Agege,9,9,9,9,9,9
Ajangbadi ikemba,23,23,23,23,23,23


In [29]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
# print out the list of categories
venues_df['VenueCategory'].unique()

There are 153 uniques categories.


array(['Sushi Restaurant', 'Fast Food Restaurant', 'Bus Station',
       'Salon / Barbershop', 'Convenience Store', 'Metro Station',
       'Burger Joint', 'Steakhouse', 'Park', 'Photography Studio',
       'Baseball Stadium', 'Soccer Field', 'Convention Center', 'Lounge',
       'Plaza', 'Fried Chicken Joint', 'Market', 'Pizza Place',
       'Shopping Mall', 'Pharmacy', 'Airport', 'Automotive Shop',
       'Harbor / Marina', 'Flea Market', 'Train Station',
       'American Restaurant', 'Wine Bar', 'Moving Target', 'Stadium',
       'Hotel', 'Food', 'Vineyard', 'Department Store', 'Beach',
       'Water Park', 'Boat or Ferry', 'Bar', 'African Restaurant',
       'Electronics Store', 'Historic Site', 'Auto Workshop',
       'Music Venue', 'Beer Garden', 'Café', 'Sporting Goods Shop',
       'Museum', 'Campground', 'Gym', 'Cupcake Shop', 'Nightclub',
       'Movie Theater', 'Bagel Shop', 'Basketball Court', 'Optical Shop',
       'Food Truck', 'Liquor Store', 'Restaurant', 'Asian Restaur

In [38]:
#getting the restaurants in the venues
res_df = venues_df[venues_df['VenueCategory'].str.contains("Restaurant")] 
res_df

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Eti Osa,6.466680,3.583260,kojima kyoto,6.472799,3.568132,Sushi Restaurant
1,Eti Osa,6.466680,3.583260,Domino's pizza Ajah,6.469620,3.586121,Fast Food Restaurant
2,Eti Osa,6.466680,3.583260,Tripps,6.466426,3.588519,Fast Food Restaurant
17,Shibiri Ekune,6.454710,3.388760,Sweet Sensation,6.453180,3.391035,Fast Food Restaurant
23,Shibiri Ekune,6.454710,3.388760,Chicken Republic,6.453549,3.385363,Fast Food Restaurant
25,Shibiri Ekune,6.454710,3.388760,de honey,6.460475,3.394775,Fast Food Restaurant
36,Olojo,6.458053,3.203222,Tantalizers alaba outlet,6.467002,3.195505,American Restaurant
43,Okokomaiko,6.464210,3.182200,Tantalizers alaba outlet,6.467002,3.195505,American Restaurant
57,Ajangbadi ikemba,6.454710,3.388760,Sweet Sensation,6.453180,3.391035,Fast Food Restaurant
63,Ajangbadi ikemba,6.454710,3.388760,Chicken Republic,6.453549,3.385363,Fast Food Restaurant


In [62]:
#to check how many restaurants per neighborhood
res_df1=res_df[['Neighborhood','VenueCategory']]
resdf2=res_df1.groupby(["Neighborhood"]).count()
print(resdf2)

                          VenueCategory
Neighborhood                           
Abarenji Street                       3
Abuke Ijeha                           2
Abule Okuta                           1
Abule abon                            3
Adeniran Ogunsanya                    5
Agege                                 3
Ajangbadi ikemba                      3
Ajao Estate                           5
Allen                                 7
Ayodele                               4
Badore                                2
Broad st                              5
Bungalow Estate                       1
Dolphin Estate                        6
Dopemu                                4
Ebute meta                            2
Ebute meta west                       2
Egan                                  1
Eti Osa                               3
Eyita                                 1
Ikorodu                               1
Oba ayangunris                        1
Okokomaiko                            1


In [39]:
# save the DataFrame as CSV file
res_df.to_csv("resdf.csv", index=False)

## 6. Analyzing Number of restaurants in each Neighborhood

In [68]:
# binning the values of restaurants in each neigborghood to low,moderate,high
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40]
labels = ['EL','VL','L','ML','MH','H','VH','EH']
resdf2['binned'] = pd.cut(resdf2['VenueCategory'], bins=bins, labels=labels)
resdf2

,VenueCategory,binned
Neighborhood,,
Abarenji Street,3,EL
Abuke Ijeha,2,EL
Abule Okuta,1,EL
Abule abon,3,EL
Adeniran Ogunsanya,5,EL
Agege,3,EL
Ajangbadi ikemba,3,EL
Ajao Estate,5,EL
Allen,7,VL


## 7. Conclusion

Observations:

Alot of restaurants are concentrated on the islan in lagos state with about 35 in victoria islan which is the higest number in our analysis, 27 in marina another neigborhood on the island of lagos state, the next neigborhood with very high number of restaurants is obalende which is just about 5 minute drive from lagos island, so it suffice to say that from the analysis above, restaurants are concentrated in the islands of lagos and its environs this is due to the fact that the islands of lagos is the most affluent neighborhoods of lagis state and majority of business districts, headquarter of large businesses in Nigeria are located on the islands. 

on the other hand moderate number of restaurants are in opebi and its environs, opebi which is situeted in Ikeja the capital city of Lagos state, this number of restaurants is expected as ikeja houses some affluent locations and expensive destinations also. whereas alot of other neigborhoods have very low to low number of restaurants, infact some streets in lagos have one restaurant, these are the potential areas for investors and real estate managers to build prospective restaurants, neigborhoods like Eyita, Ikorodu,Olojo amongst others.